# getting data from the one minute dataset

# rollong mean of the volume

# select which stock to be used in the analysis

Top Trading Volume Stocks & AAPL & MGM & FITB & MPC & GS \\
Bottom Trading Volume Stocks & AEP & HP & KLAC & GPN & ROK \\

In [ ]:
lst = ['AAPL', 'MGM','FITB','MPC','GS','AEP','HP','KLAC','GPN','ROK']
for symbol in lst:

In [ ]:
%%bash
docker build --build-arg UID=$UID --build-arg GIT_TOKEN=$GIT_TOKEN -t d345fn01 .
docker run --name d345fn01 --user $(id -u) -v $(pwd):/homes/80/kang -it --gpus '"device=3,4,5"' d345fn01
docker run -it -v /scratch/local/kang:/data d345fn01

In [ ]:
%%bash
cd /data
touch run12feb.sh
cat > run12feb.sh << 'EOF'
input_dir="/data/" # Directory containing .7z files
sp500_dir="/homes/80/kang/SP500/" # Directory to store all stock data directories 

# Define a list of stock names
declare -a stock_list=('AAPL' 'MGM' 'FITB' 'MPC' 'GS' 'AEP' 'HP' 'KLAC' 'GPN' 'ROK')

# Iterate over each .7z file in the input directory
for file in "$input_dir"/*.7z; do
    # Extract the stock name from the file name
    stock_name=$(echo "$file" | grep -oP '(?<=__)\w+(?=_)')

    # Check if the extracted stock name is in the list
    if [[ " ${stock_list[@]} " =~ " ${stock_name} " ]]; then
        # Create directories for the stock data and its subdirectories
        stock_dir="${sp500_dir}${stock_name}_data"
        mkdir -p "$stock_dir/Book_10" "$stock_dir/Flow_10"

        # Unzip the file into a temporary directory
        temp_dir=$(mktemp -d)
        7z x "$file" -o"$temp_dir"

        # Move files to appropriate subdirectories
        for csv_file in "$temp_dir"/*.csv; do
            if [[ $csv_file == *"orderbook"* ]]; then
                mv "$csv_file" "$stock_dir/Book_10/"
            elif [[ $csv_file == *"message"* ]]; then
                mv "$csv_file" "$stock_dir/Flow_10/"
            fi
        done

        # Remove the temporary directory
        rm -r "$temp_dir"

        # Move the stock directory into the SP500 directory
        # mv "$stock_dir" "$sp500_dir/" # This line is not needed since "$stock_dir" already includes "$sp500_dir"
    fi
done
EOF



In [5]:
%%bash
cd /homes/80/kang/SP500/
ls -lh

total 76K
drwxr-xr-x 4 kang        3509  82 Dec  7 15:40 AAL_data
drwxr-xr-x 4 kang        3509  82 Dec  6 06:40 AAP_data
drwxr-xr-x 4 kang        3509  48 Nov 27 14:19 AAPL_data
drwxr-xr-x 4 kang        3509  48 Nov 27 14:30 ABBV_data
drwxr-xr-x 4 kang        3509  82 Dec  7 09:09 ABC_data
drwxr-xr-x 4 kang        3509  48 Nov 27 14:33 ABMD_data
drwxr-xr-x 4 kang        3509  48 Nov 27 14:33 ABT_data
drwxr-xr-x 4 kang        3509  82 Dec  7 04:01 ACN_data
drwxr-xr-x 4 kang        3509  48 Nov 27 14:36 ADBE_data
drwxr-xr-x 4 kang users        82 Jan 22 12:45 BAC_data
drwxr-xr-x 4 kang        3509  48 Dec 10 03:01 CHD_data
drwxr-sr-x 4 kang flair-users  48 Dec  7 09:31 CLX_data
-rw-r--r-- 1 kang        3509 73K Dec  4 07:27 state_arrays_kang.pkl


In [ ]:
(base) kang@flair-node-01:/scratch/local/kang/SP500$ du -lh
7.6G    ./AEP_data/Book_10
1.6G    ./AEP_data/Flow_10
9.2G    ./AEP_data
7.0G    ./GS_data/Book_10
1.4G    ./GS_data/Flow_10
8.4G    ./GS_data
16G     ./FITB_data/Book_10
3.0G    ./FITB_data/Flow_10
19G     ./FITB_data
0       ./HP_data/Book_10
0       ./HP_data/Flow_10
0       ./HP_data
2.7G    ./GPN_data/Book_10
579M    ./GPN_data/Flow_10
3.3G    ./GPN_data
0       ./KLAC_data/Book_10
0       ./KLAC_data/Flow_10
0       ./KLAC_data
54G     ./AAPL_data/Book_10
11G     ./AAPL_data/Flow_10
64G     ./AAPL_data
0       ./MGM_data/Book_10
0       ./MGM_data/Flow_10
0       ./MGM_data
0       ./MPC_data/Book_10
0       ./MPC_data/Flow_10
0       ./MPC_data
0       ./ROK_data/Book_10
0       ./ROK_data/Flow_10
0       ./ROK_data
103G    .

In [1]:
lst = ['AEP','GS','FITB','GPN','AAPL']

In [2]:
path = '/scratch/local/kang/SP500'

## vwap file func

In [3]:
import pandas as pd
import numpy as np
def data_alignment(ATFolder):
    '''
    The logic here is to load the three dir: messages data, rolling mean and vwap
    Then find the common_dates of all these three,
    Then pass into the init of the base env, 
    To only load the data window in the common dates.
    But we should also align the symbols, it has been done, as we will pass symbols 
    into the load_forecasted_and_original_volume_VWAP and load_forecasted_volume_RM
    '''
    def load_forecasted_and_original_volume_VWAP(symbol):
        import pandas as pd
        import numpy as np
        dir = '/homes/80/kang/cmem/output/0900_r_output_with_features_csv_fractional_shares_clipped_vwap/'
        df = pd.read_csv(dir+f'{symbol}.csv',index_col=0)
        df['symbol'] = symbol
        from datetime import datetime, timedelta
        timeHMs = np.array([int((datetime(2023, 1, 1, 9, 30) + i * timedelta(minutes=15)).strftime('%H%M')) for i in range(26)])
        timeHMs = np.tile(timeHMs, (1, int(df.shape[0]/timeHMs.shape[0]))).squeeze()
        df['timeHMs'] = timeHMs
        df = df[['date', 'timeHMs', 'x', 'qty', 'symbol']]
        d = df[['date', 'timeHMs', 'x']].pivot(index = 'date', columns = 'timeHMs')
        f = df[['date', 'timeHMs', 'qty']].pivot(index = 'date', columns = 'timeHMs')
        return d, f # x, qty

    def load_forecasted_volume_RM(symbol):
        import pandas as pd
        import numpy as np
        dir = '/homes/80/kang/cmem/data/01_raw_rolling_mean_15min_bin/'
        df = pd.read_csv(dir+f'{symbol}.csv',index_col=0)
        df.columns = ['date', 'timeHMs', 'timeHMe', 'x_rm', 'symbol']
        df.date = df.date.apply(lambda d: str(d)[:4]+'-'+str(d)[4:6]+'-'+str(d)[6:8]) if len(str(df.date[0])) == 8 else df.date
        d = df[['date', 'timeHMs', 'x_rm']].pivot(index = 'date', columns = 'timeHMs')
        d = d.fillna(method = "ffill")
        return d # x

    def load_files(symbol):
        alphatradePath = ATFolder
        # alphatradePath = f"{ATFolder}/{symbol}_data"
        messagePath = alphatradePath+"/Flow_10/"
        orderbookPath = alphatradePath+"/Book_10/"
        from os import listdir; from os.path import isfile, join; import pandas as pd
        readFromPath = lambda data_path: sorted([f for f in listdir(data_path) if isfile(join(data_path, f))])
        messageFiles, orderbookFiles = readFromPath(messagePath), readFromPath(orderbookPath)
        message_dates = np.array([m[4:14] for m in messageFiles])
        message_dates = np.array([m.split("_")[1] for m in messageFiles])
        return message_dates
    
    def get_symbols():
        from os.path import isdir
        from os import listdir
        path_dir1 = '/homes/80/kang/cmem/output/0900_r_output_with_features_csv_fractional_shares_clipped_vwap/'
        path_dir2 = ATFolder
        symbols1 = np.array([dirname.split(".")[0] for dirname in listdir(path_dir1)])
        symbols2 = np.array([path_dir2.split("_")[0].split("/")[-1]])
        # symbols2 = np.array([dirname.split("_")[0] for dirname in listdir(path_dir2) 
        #             if isdir(os.path.join(path_dir2, dirname))])
        intersection = np.intersect1d(symbols1, symbols2)
        return intersection
    common_stocks = get_symbols()
    assert len(common_stocks) >= 1
    
    def get_raw_VWAPs_ORACLEs_RMs():
        ds = [(symbol, *load_forecasted_and_original_volume_VWAP(symbol)) for symbol in common_stocks]
        VWAPs = [(d[0], d[1]) for d in ds]
        ORACLEs = [(d[0], d[2]) for d in ds]
        RMs = [(symbol, load_forecasted_volume_RM(symbol)) for symbol in common_stocks]
        return VWAPs, ORACLEs, RMs
    VWAPs, ORACLEs, RMs = get_raw_VWAPs_ORACLEs_RMs()
    
    def get_common_dates():
        dates_vwap =  VWAPs[-1][-1].index.to_numpy()
        dates_rm =  RMs[-1][-1].index.to_numpy()
        message_dates = load_files(common_stocks[-1]) 
        common_dates = np.intersect1d(np.intersect1d(dates_vwap, message_dates),dates_rm)
        return common_dates
    common_dates = get_common_dates()
    
    def align_dates(Xs):
        Xs = {d[0]: d[1].loc[common_dates] for d in Xs}
        return Xs
    VWAPs, ORACLEs, RMs = list(map(align_dates, [VWAPs, ORACLEs, RMs]))

    def generate_TWAPs(RMs):
        def f(df):
            dff = df.copy()
            dff.loc[:, :] = 1
            return dff
        return {symbol: f(df) for symbol, df in RMs.items()}
    TWAPs = generate_TWAPs(RMs)
            
    return common_dates, common_stocks, VWAPs, ORACLEs, RMs, TWAPs

lst = ['AEP','GS','FITB','GPN','AAPL']
symbol = 'AEP'
ATFolder = f"/scratch/local/kang/SP500/{symbol}_data"
# ATFolder = f"/homes/80/kang/SP500/{symbol}_data"
# ATFolder = f"/homes/80/kang/SP500/ABC_data"
# ATFolder = f"/homes/80/kang/SP500/ACN_data"
# ATFolder = f"/homes/80/kang/SP500/AAP_data"
common_dates, common_stocks, VWAPs, ORACLEs, RMs, TWAPs = data_alignment(ATFolder)

In [11]:
common_stocks, common_dates

(array(['AEP'], dtype='<U4'),
 array(['2017-07-20', '2017-07-21', '2017-07-24', '2017-07-25',
        '2017-07-26', '2017-07-27', '2017-07-28', '2017-07-31',
        '2017-08-01', '2017-08-02', '2017-08-03', '2017-08-04',
        '2017-08-07', '2017-08-08', '2017-08-09', '2017-08-10',
        '2017-08-11', '2017-08-14', '2017-08-15', '2017-08-16',
        '2017-08-17', '2017-08-18', '2017-08-21', '2017-08-22',
        '2017-08-23', '2017-08-24', '2017-08-25', '2017-08-28',
        '2017-08-29', '2017-08-30', '2017-08-31', '2017-09-01',
        '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08',
        '2017-09-11', '2017-09-12', '2017-09-13', '2017-09-14',
        '2017-09-15', '2017-09-18', '2017-09-19', '2017-09-20',
        '2017-09-21', '2017-09-22', '2017-09-25', '2017-09-26',
        '2017-09-27', '2017-09-28', '2017-09-29', '2017-10-02',
        '2017-10-03', '2017-10-04', '2017-10-05', '2017-10-06',
        '2017-10-10', '2017-10-11', '2017-10-12', '2017-10-13',
        '2

In [4]:
lstsymbols = ['AEP','GS','FITB','GPN','AAPL']

# run r to get the cmem results

In [5]:
cd /homes/80/kang/cmem/data_minmax/02.2_data_r_input_kf_minmax/

/homes/80/kang/cmem/data_minmax/02.2_data_r_input_kf_minmax


In [6]:
path = '/homes/80/kang/cmem/data_minmax/02.2_data_r_input_kf_minmax/'
from os import listdir
lst = listdir(path)
lstt = [file[:-4] for file in lst]

In [7]:
for idx, itm in enumerate(lstt):
    if itm in lstsymbols:
        print(idx, itm)

3 AAPL
16 AEP
174 FITB
196 GPN
199 GS


## use this r code:
/homes/80/kang/cmem/rkf_fraction.r
## hers is the output dir
/homes/80/kang/cmem/output/0400_r_kl_output_raw_data_minmax/


## check whether all the data is generated correctly

In [8]:
%%bash
cd /homes/80/kang/cmem/output/0400_r_kl_output_raw_data_minmax/
ls | wc

    472     472    3833


In [9]:
#  note the data might not all generated

# we have to generate this after the r code output: 
0900_r_output_with_features_csv_fractional_shares_clipped_vwap

## get symbol  codes

In [5]:
dir = '/homes/80/kang/cmem/output/0400_r_kl_output_raw_data_minmax/'
import os 
from os import listdir
lst = sorted([file[:-4] for file in listdir(dir)])
len(lst)

472

In [13]:
file_path = '/homes/80/kang/cmem/stock_names.txt'
reloaded_stock_names = []
with open(file_path, 'r') as f:
    reloaded_stock_names = [line.strip() for line in f.readlines()]
reloaded_stock_names,len(reloaded_stock_names)
lst = reloaded_stock_names

### from mihai's shares outstanding data

In [14]:
from os import listdir
from os.path import isfile, join
dir0 = "/homes/80/kang/2017daily_csv/"
onlyFiles = sorted([f for f in listdir(dir0) if isfile(join(dir0, f))])
dfflst =[]
i=0
# for i in range(len(onlyFiles)):
print(i)
name = onlyFiles[i]
import pandas as pd
# name ="CMS.csv"

dir = dir0+name
df = pd.read_csv(dir)
mhlst = sorted(df.ticker.to_list())
len(mhlst)
lstt = [file for file in mhlst if file in lst]
len(lstt)

0


384

In [15]:
lst0=[file for file in lst if file not in lstt]
len(lst0)

99

### lst0 is the missing parts

In [16]:
lst0

['A',
 'AAPL',
 'ADBE',
 'ADSK',
 'AKAM',
 'ALGN',
 'ALXN',
 'AMAT',
 'AMGN',
 'AMZN',
 'ANSS',
 'ATVI',
 'AVGO',
 'BF.B',
 'BIIB',
 'BRK.B',
 'CBOE',
 'CDW',
 'CERN',
 'CHRW',
 'CHTR',
 'CINF',
 'CMCSA',
 'COST',
 'CPRT',
 'CSCO',
 'CTAS',
 'CTSH',
 'CTXS',
 'DISCA',
 'DISCK',
 'DISH',
 'DLTR',
 'EA',
 'EBAY',
 'EQIX',
 'EXPD',
 'EXPE',
 'FANG',
 'FAST',
 'FFIV',
 'FISV',
 'FITB',
 'FLIR',
 'FTNT',
 'GILD',
 'GOOG',
 'GOOGL',
 'GRMN',
 'HBAN',
 'HOLX',
 'HSIC',
 'IDXX',
 'ILMN',
 'INTC',
 'INTU',
 'IPGP',
 'ISRG',
 'JBHT',
 'JKHY',
 'KHC',
 'KLAC',
 'LKQ',
 'LRCX',
 'MCHP',
 'MKTX',
 'MNST',
 'MSFT',
 'MXIM',
 'NDAQ',
 'NFLX',
 'NTAP',
 'NTRS',
 'NVDA',
 'NWSA',
 'ODFL',
 'ORLY',
 'PAYX',
 'PBCT',
 'PCAR',
 'PYPL',
 'QCOM',
 'QRVO',
 'REGN',
 'ROST',
 'SBAC',
 'SBUX',
 'SIVB',
 'SNPS',
 'SWKS',
 'TRIP',
 'TROW',
 'TSCO',
 'TTWO',
 'ULTA',
 'VRSK',
 'VRSN',
 'VRTX',
 'WYNN']